# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

In [4]:
wb = xw.Book('себестоимостьА_в1.xlsx')
recipes = wb.sheets['Рецептура']
c = recipes.range('G7:O10').options(np.array).value
c = np.nan_to_num(c)
unit_prices = recipes.range('G14:O14').value
cost = np.sum(c*unit_prices, axis=1)
cost

array([21.48 , 16.525, 17.423, 18.085])

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

In [5]:
recipes.range('T6').value = 'Себестоимость'
recipes.range('T7').options(transpose=True).value = cost

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

In [6]:
recipes.range('T4:T6').api.Merge()
recipes.range('T4:T6').font.bold = True
recipes.range('T5').columns.autofit()

4. Выполнить 3 с помощью "протягиваемых" формул.

In [7]:
wb1 = xw.Book('себестоимостьА_в1.xlsx')
recipes1 = wb1.sheets['Рецептура']
recipes1.range('Q7:Q10').formula = f'=$G$14*G7+$H$14*H7+$I$14*I7+$J$14*J7+$K$14*K7+$L$14*L7+M7*$M$14+N7*$N$14+O7*$O$14'

## Лабораторная работа 7.1

In [3]:
import csv
import numpy as np
import pandas as pd
import xlwings as xw
from xlwings.constants import AutoFillType

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [9]:
reviews = pd.read_csv('reviews_sample.csv', delimiter=',')
reviews.rename(columns = {'Unnamed: 0':'id'}, inplace = True )
reviews

,id,user_id,recipe_id,date,rating,review
0,370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
1,624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
2,187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
3,706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
4,312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...
...,...,...,...,...,...,...
126691,1013457,1270706,335534,2009-05-17,4,This recipe was great! I made it last night. I...
126692,158736,2282344,8701,2012-06-03,0,This recipe is outstanding. I followed the rec...
126693,1059834,689540,222001,2008-04-08,5,"Well, we were not a crowd but it was a fabulou..."
126694,453285,2000242659,354979,2015-06-02,5,I have been a steak eater and dedicated BBQ gr...


In [10]:
recipes = pd.read_csv('recipes_sample.csv', delimiter=',')[['id', 'name', 'minutes', 'submitted', 'description', 'n_ingredients']]
recipes

,id,name,minutes,submitted,description,n_ingredients
0,44123,george s at the cove black bean soup,90,2002-10-25,an original recipe created by chef scott meska...,18.0
1,67664,healthy for them yogurt popsicles,10,2003-07-26,my children and their friends ask for my homem...,NaN
2,38798,i can t believe it s spinach,30,2002-08-29,"these were so go, it surprised even me.",8.0
3,35173,italian gut busters,45,2002-07-27,my sister-in-law made these for us at a family...,NaN
4,84797,love is in the air beef fondue sauces,25,2004-02-23,i think a fondue is a very romantic casual din...,NaN
...,...,...,...,...,...,...
29995,267661,zurie s holey rustic olive and cheddar bread,80,2007-11-25,this is based on a french recipe but i changed...,10.0
29996,386977,zwetschgenkuchen bavarian plum cake,240,2009-08-24,"this is a traditional fresh plum cake, thought...",11.0
29997,103312,zwiebelkuchen southwest german onion cake,75,2004-11-03,this is a traditional late summer early fall s...,NaN
29998,486161,zydeco soup,60,2012-08-29,this is a delicious soup that i originally fou...,NaN


2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [11]:
five_per_rev=pd.DataFrame(reviews.sample(n=round(reviews.shape[0]/100*5)))
five_per_rec=pd.DataFrame(recipes.sample(n=round(recipes.shape[0]/100*5)))
five_per_rev

,id,user_id,recipe_id,date,rating,review
16522,442386,1140356,199332,2009-02-21,5,I tried this recipe using part white flour and...
18622,1074817,728020,135350,2013-09-12,4,The only thing I would do next time is mix in ...
67929,1123751,453828,57679,2009-05-31,5,"WOW, truly a winner! I don't like key lime pi..."
58332,242915,143288,17937,2004-05-28,2,I was disappointed at how faint the pineapple ...
44658,971213,591651,415308,2010-03-07,5,We've made this recipe for years. We substitut...
...,...,...,...,...,...,...
108892,257503,438003,53730,2008-11-11,5,This was great and easy!!!
53039,784120,1643478,137701,2010-06-30,0,Here's a great idea! Add a few walnuts in wit...
10003,837711,383346,110683,2009-12-28,5,It was my first time doing waffles and I'm gla...
35943,244364,107583,134422,2008-08-24,0,"Unfortunately, I had the same problems as the ..."


In [12]:
five_per_rec

,id,name,minutes,submitted,description,n_ingredients
13368,222914,hasty cobbler,45,2007-04-16,crust starts on the bottom and rises to the to...,6.0
17078,44923,market street clam chowder,60,2002-10-29,this is an excellent chowder recipe from a res...,NaN
6561,307608,chocolate cool whip pie,10,2008-06-05,if you can use the microwave then you can make...,3.0
27960,225325,traditional southern greens mustard turnip o...,205,2007-04-29,"my family's way of ""fixin'""greens. you can use...",5.0
20120,385996,peach crumble crunch,45,2009-08-18,any stewed fruit could be used here.,8.0
...,...,...,...,...,...,...
29187,51217,wheat germ brownies,45,2003-01-14,these have a slightly caramel crunchy bottom l...,11.0
21787,144223,pumpkin walnut praline bars,80,2005-11-07,these bars have a pumpkin pie filling and are ...,12.0
17234,249354,mean green chicken enchiladas,80,2007-08-28,if it's green it's got to be good ;)\r\n\r\nus...,18.0
27211,164727,thai beef crock pot,255,2006-04-17,delicious and full of flavor.\r\n\r\nfrom bett...,NaN


In [13]:
with pd.ExcelWriter("recipes.xlsx",engine='xlsxwriter') as writer:
    five_per_rec.to_excel(writer, sheet_name="Рецепты")
    five_per_rev.to_excel(writer, sheet_name = "Отзывы")
  

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [15]:
import xlwings as xw
book = xw.Book('recipes.xlsx')
sheet = book.sheets['Рецепты']
sheet.range("H1").value = 'seconds_assign'
second_assign = (five_per_rec["minutes"] * 60).to_numpy()
sheet.range("H2:H1501").options(transpose=True).value = second_assign

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [16]:
sheet.range("I1").value = 'seconds_formula'
data = sheet.range("I2").formula = f'=D2 * 60'
xw.Range("I2:I1501").formula = data

5. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [17]:
sheet.range('H1').api.Font.Bold = True
sheet.range('I1').api.Font.Bold = True
sheet.range('H1').api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter
sheet.range('I1').api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter

6. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [18]:
minutes = sheet.range("D2:D1501").value
for i in range(len(minutes)):
    if minutes[i] < 5:
        sheet.range(f'D{i+2}').color = (0, 255, 0)
    elif 5 <= minutes[i] <= 10:
        sheet.range(f'D{i+2}').color = (255, 255, 0)
    else:
        sheet.range(f'D{i+2}').color = (255, 0, 0)

7. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [20]:
formula = sheet.range("J2").formula = f'=COUNTIF(Отзывы!$C$2:$C${len(five_per_rev)},C2)'
sheet.range("J1").value = 'n_reviews'
xw.Range("J2:J1501").formula = formula

## Лабораторная работа 7.2

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

9. В файле `recipes_model.csv` находится модель данных предметной области "рецепты". При помощи пакета `csv` считайте эти данные. При помощи пакета `xlwings` запишите данные на лист `Модель` книги `recipes_model.xlsx`, начиная с ячейки `A2`, не используя циклы. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука. 

10. При помощи пакета `xlwings` добавьте в столбец J формулу для описания столбца на языке SQL. Формула должна реализовывать следующую логику:

    1\. в начале строки идут значения из столбцов В и C (значение столбца С приведено к верхнему регистру), разделенные пробелом
    
    2\. далее идут слова на основе столбца "Ключ"
        2.1 если в столбце "Ключ" указано значение "PK", то дальше через пробел идет ключевое слово "PRIMARY KEY"
        2.2 если в столбце "Ключ" указано значение "FK", то дальше через пробел идет ключевое слово "REFERENCES", затем значения столбцов H и I в формате "название_таблицы(название_столбца)"
        
    3\. если в столбце "Обязательно к заполнению" указано значение "Y" и в столбце "Ключ" указано не "PK", то дальше через пробел идет ключевое слово "NOT NULL".

Заполните этой формулой необходимое количество строк, используя "протягивание". Количество строк для протягивания определите на основе данных.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

11. При помощи пакета `xlwings` измените стилизацию листа `Модель`.
* для заголовков добавьте заливку цвета `00ccff`
* примените автоподбор ширины столбца;
* сделайте шрифт заголовков полужирным;
* добавьте таблице автофильтр.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

12. Посчитайте количество атрибутов для каждой из сущностей. Создайте лист `Статистика` и запишите в него результат группировки, начиная с ячейки "А1". Визуализируйте полученный результат при помощи столбчатой диаграммы. Сохраните полученную визуализацию на лист `Статистика`, начиная с ячейки "E2".  Сделайте скриншот листа `Статистика` и прикрепите в ячейку ноутбука.

* Вы можете воспользоваться методами для визуализации, которые поставляются вместе с объектами `pandas` (см. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot) 